In [1]:
from IPython.display import display, HTML
display(HTML("""
<style>
div.container{width:85% !important;}
div.cell.code_cell.rendered{width:100%;}
div.input_prompt{padding:0px;}
div.CodeMirror {font-family:Consolas; font-size:12pt;}
div.output {font-size:12pt; font-weight:bold;}
div.input {font-family:Consolas; font-size:12pt;}
div.prompt {min-width:70px;}
div#toc-wrapper{padding-top:120px;}
div.text_cell_render ul li{font-size:12pt;padding:5px;}
table.dataframe{font-size:12px;}
</style>
"""))

<b><font size="7" color="red">ch14. 웹데이터수집1_정적_공공api</font></b>
# 1절. BeautifulSoup과 parser
`pip install bs4` 아나콘다를 설치하면 자동 설치되는 패키지 7500개에 포함
- 공식 Documentation : https://beautiful-soup-4.readthedocs.io/en/latest/

In [2]:
# 내 local PC의 파일을 url처럼 접근
import requests #http 요청 처리하는 lib
from requests_file import FileAdapter #file://프로토콜을 다루기 위함

In [3]:
s = requests.Session() #HTTP 요청처리하는 세션 객체
s.mount("file://", FileAdapter())
# file://경롤로 들어오면 이 요청은 c:(로컬파일)을 접근
response = s.get('file:///ai/source/01_python/data/ch14_sample.html')
response                    

<Response [200]>

In [4]:
if response:
    print('잘 접근하였습니다.')
else:
    print('접근을 못했습니다.')

잘 접근하였습니다.


In [5]:
response.status_code
    # 200 : 정상
    # 404 : 없는페이지
    # 406:  get, post 오류

200

In [6]:
response.content # 바리너리 형식의 html 내용

b'<!DOCTYPE html>\r\n<html lang="en">\r\n<head>\r\n  <meta charset="UTF-8">\r\n</head>\r\n<body>\r\n  <h1 class="greeting css" id="text">Hello, CSS</h1>\r\n  <h1 class="css">Hi, CSS</h1>\r\n  <div id="subject">subject \xec\x84\xa0\xed\x83\x9d\xec\x9e\x90 \xec\x95\x88\xec\x9d\x98 \xeb\x82\xb4\xec\x9a\xa9</div>\r\n  <p>CSS \xec\x84\xa0\xed\x83\x9d\xec\x9e\x90\xeb\x8a\x94 \xeb\x8b\xa4\xec\x96\x91\xed\x95\x9c \xea\xb3\xb3\xec\x97\x90\xec\x84\x9c \xed\x99\x9c\xec\x9a\xa9\xeb\x90\xa9\xeb\x8b\x88\xeb\x8b\xa4</p>\r\n  <div class="contents">\r\n    \xec\x84\xa0\xed\x83\x9d\xec\x9e\x90\xeb\xa5\xbc \xec\x96\xb4\xeb\x96\xbb\xea\xb2\x8c \xec\x9e\x91\xec\x84\xb1\xed\x95\x98\xeb\x8a\x90\xeb\x83\x90\xec\x97\x90 \xeb\x94\xb0\xeb\x9d\xbc\r\n    <span>\xeb\x8b\xa4\xeb\xa5\xb8<b>\xec\x9a\x94\xec\x86\x8c\xea\xb0\x80 \xeb\xb0\x98\xed\x99\x98</b></span>\xeb\x90\xa9\xeb\x8b\x88\xeb\x8b\xa4\r\n  </div>\r\n  <div>CSS \xec\x84\xa0\xed\x83\x9d\xec\x9e\x90\xeb\x8a\x94 \xeb\x8b\xa4\xec\x96\x91\xed\x95\x9c \xea\xb3\

In [7]:
print(response.content.decode('utf-8'))

<!DOCTYPE html>
<html lang="en">
<head>
  <meta charset="UTF-8">
</head>
<body>
  <h1 class="greeting css" id="text">Hello, CSS</h1>
  <h1 class="css">Hi, CSS</h1>
  <div id="subject">subject 선택자 안의 내용</div>
  <p>CSS 선택자는 다양한 곳에서 활용됩니다</p>
  <div class="contents">
    선택자를 어떻게 작성하느냐에 따라
    <span>다른<b>요소가 반환</b></span>됩니다
  </div>
  <div>CSS 선택자는 다양한 곳에 <b>활용</b>됩니다</div>
</body>
</html>


In [8]:
response.text # HTML 파일의 텍스트 내용

'<!DOCTYPE html>\r\n<html lang="en">\r\n<head>\r\n  <meta charset="UTF-8">\r\n</head>\r\n<body>\r\n  <h1 class="greeting css" id="text">Hello, CSS</h1>\r\n  <h1 class="css">Hi, CSS</h1>\r\n  <div id="subject">subject 선택자 안의 내용</div>\r\n  <p>CSS 선택자는 다양한 곳에서 활용됩니다</p>\r\n  <div class="contents">\r\n    선택자를 어떻게 작성하느냐에 따라\r\n    <span>다른<b>요소가 반환</b></span>됩니다\r\n  </div>\r\n  <div>CSS 선택자는 다양한 곳에 <b>활용</b>됩니다</div>\r\n</body>\r\n</html>'

In [9]:
# html 파싱
from bs4 import BeautifulSoup
soup = BeautifulSoup(response.content, # response.text
                    "html.parser")
# soup

In [10]:
# soup.select_one('선택자') : 해당 선택자 처음 하나만
el = soup.select_one('h1') # 처음 나오는 h1태그 하나만
print('el : ', el)
print('el.text :' ,el.string)
print('el의 속성들 :', el.attrs)
print('el의 id속성 :', el.attrs['id'])
print('el의 id속성 :', el.attrs.get)
print('el의 href속성 :', el.attrs.get('href'))
print('el의 태그이름 :', el.name)

el :  <h1 class="greeting css" id="text">Hello, CSS</h1>
el.text : Hello, CSS
el의 속성들 : {'class': ['greeting', 'css'], 'id': 'text'}
el의 id속성 : text
el의 id속성 : <built-in method get of dict object at 0x000001626B0BF3C0>
el의 href속성 : None
el의 태그이름 : h1


In [11]:
# soup.select('선택자') : 해당 선택자 모두
els = soup.select('h1') # h1태그를  list
print('els :' ,els)
print('els의 text들 :', [el.text for el in els])
print('els의 string들 :', [el.string for el in els])
print('els의 속성들 :', [el.attrs for el in els])
print('els의 class 속성들 :', [el.attrs.get('class') for el in els])

els : [<h1 class="greeting css" id="text">Hello, CSS</h1>, <h1 class="css">Hi, CSS</h1>]
els의 text들 : ['Hello, CSS', 'Hi, CSS']
els의 string들 : ['Hello, CSS', 'Hi, CSS']
els의 속성들 : [{'class': ['greeting', 'css'], 'id': 'text'}, {'class': ['css']}]
els의 class 속성들 : [['greeting', 'css'], ['css']]


In [12]:
# soup.find(태그, 속성) : soup.select_one('선택자')와 유사
print('select_one :', soup.select_one('h1.css'))
print('find :', soup.find('h1',{'class':'css'}) )# soup.select_one(h1.css)
print('find :', soup.find('h1',class_='css') )
print()
print('select_one :', soup.select_one('h1#textr'))
print('find :', soup.find('h1',{'id':'text'}))

select_one : <h1 class="greeting css" id="text">Hello, CSS</h1>
find : <h1 class="greeting css" id="text">Hello, CSS</h1>
find : <h1 class="greeting css" id="text">Hello, CSS</h1>

select_one : None
find : <h1 class="greeting css" id="text">Hello, CSS</h1>


In [13]:
# soup.find_all(태그, 속성) : soup.select_one('선택자')와 유사
print('find :', soup.find('h1',{'class':'css'}) )
print('select :', soup.select('h1.css'))
print('find_all :', soup.find_all('h1', class_='css'))
print('find_all :', soup.find_all('h1', {'class':'css'}))

find : <h1 class="greeting css" id="text">Hello, CSS</h1>
select : [<h1 class="greeting css" id="text">Hello, CSS</h1>, <h1 class="css">Hi, CSS</h1>]
find_all : [<h1 class="greeting css" id="text">Hello, CSS</h1>, <h1 class="css">Hi, CSS</h1>]
find_all : [<h1 class="greeting css" id="text">Hello, CSS</h1>, <h1 class="css">Hi, CSS</h1>]


In [14]:
print('select :', soup.select('h1.css, span'))
print('find_all :', soup.find_all(['h1', 'span'], {'class':'css'}))
print('find_all :', soup.find_all('h1', class_='css') + \
                     soup.find_all('span'))

select : [<h1 class="greeting css" id="text">Hello, CSS</h1>, <h1 class="css">Hi, CSS</h1>, <span>다른<b>요소가 반환</b></span>]
find_all : [<h1 class="greeting css" id="text">Hello, CSS</h1>, <h1 class="css">Hi, CSS</h1>]
find_all : [<h1 class="greeting css" id="text">Hello, CSS</h1>, <h1 class="css">Hi, CSS</h1>, <span>다른<b>요소가 반환</b></span>]


In [15]:
# 없는 엘리먼트 찾기
print('select(빈 list) :', soup.select('a.css'))
print('select_one(None) :', soup.select_one('a.css'))
print('find_all(빈 list) :', soup.find_all('a', {'class', 'css'}))
print('find(None) :', soup.find('a', {'class':'css'}))

select(빈 list) : []
select_one(None) : None
find_all(빈 list) : []
find(None) : None


# 2절. 정적 웹 테디어 수집(정적 웹크롤링)
## 2.1 BeautifulSoup을 활용한 html 웹 데이터수집
### 1) 환율정보 가져오기(네이버증권->시장지표)
- https://finance.naver.com/marketindex/

In [16]:
# 크롤링 허용범위는 사이트마다~.robots.txt에서 확인
    #Allow : / - 사이트의 모든 경로(/)에 대한 크롤릴 허용

In [17]:
# 방법1
import requests
from bs4 import BeautifulSoup
url = 'https://finance.naver.com/marketindex/'
response = requests.get(url)
# response.status_code
# response.content.decode('cp949')
soup = BeautifulSoup(response.text, "html.parser")

In [18]:
# 방법2
from urllib.request import urlopen
url = 'https://finance.naver.com/marketindex/'
response = urlopen(url)
response.status
# response.read() / response.read().decode('cp949')
soup = BeautifulSoup(response, "html.parser")

In [19]:
# div.head_info > span.value (find함수)
prices = []
headinfos = soup.find_all('div', class_='head_info')
for headinfo in headinfos:
    price = headinfo.find('span', class_='value')
    # print(price.text.replace(',', ''))
    prices.append(float(''.join(price.text.split(',') ) ) )
print(prices)

[1452.3, 942.68, 1679.0, 203.98, 153.42, 1.1564, 1.3161, 99.47, 59.75, 1699.21, 4009.8, 190334.75]


In [20]:
# div.head_info > span.value (select함수)
prices = soup.select('div.head_info > span.value')
print([float(p.text.replace(',','')) for p in prices])

[1452.3, 942.68, 1679.0, 203.98, 153.42, 1.1564, 1.3161, 99.47, 59.75, 1699.21, 4009.8, 190334.75]


In [21]:
titles = soup.select('h3.h_lst > span.blind')
for t in titles:
    print(t.text, end='\t')

미국 USD	일본 JPY(100엔)	유럽연합 EUR	중국 CNY	달러/일본 엔	유로/달러	영국 파운드/달러	달러인덱스	WTI	휘발유	국제 금	국내 금	

In [22]:
units = soup.select('div.head_info > span > span.blind')
units = [unit.text for unit in units]
units.insert(7, '')
units

['원', '원', '원', '원', '엔', '달러', '달러', '', '달러', '원', '달러', '원']

In [23]:
statuses = soup.select('div.head_info > span.blind')
for idx in range(len(statuses)):
    print(statuses[idx].text, end='\t')

하락	하락	하락	하락	상승	하락	상승	하락	상승	상승	상승	상승	

In [24]:
len(titles), len(prices), len(units), len(statuses)

(12, 12, 12, 12)

In [25]:
for idx in range(len(titles)):
    print("{} : {}{} - {}".format(titles[idx].text,
                                 prices[idx].text,
                                 units[idx],
                                 statuses[idx].text))

미국 USD : 1,452.30원 - 하락
일본 JPY(100엔) : 942.68원 - 하락
유럽연합 EUR : 1,679.00원 - 하락
중국 CNY : 203.98원 - 하락
달러/일본 엔 : 153.4200엔 - 상승
유로/달러 : 1.1564달러 - 하락
영국 파운드/달러 : 1.3161달러 - 상승
달러인덱스 : 99.4700 - 하락
WTI : 59.75달러 - 상승
휘발유 : 1699.21원 - 상승
국제 금 : 4009.8달러 - 상승
국내 금 : 190334.75원 - 상승


In [26]:
for title, price, unit, status in zip(titles, prices, units, statuses):
    print("{} : {}{} -{}".format(title.text,
                                price.text,
                                unit,
                                status.text))

미국 USD : 1,452.30원 -하락
일본 JPY(100엔) : 942.68원 -하락
유럽연합 EUR : 1,679.00원 -하락
중국 CNY : 203.98원 -하락
달러/일본 엔 : 153.4200엔 -상승
유로/달러 : 1.1564달러 -하락
영국 파운드/달러 : 1.3161달러 -상승
달러인덱스 : 99.4700 -하락
WTI : 59.75달러 -상승
휘발유 : 1699.21원 -상승
국제 금 : 4009.8달러 -상승
국내 금 : 190334.75원 -상승


### 2)이번주 로또 번호 출력
- https://dhlottery.co.kr/gameResult.do?method=byWin  (google에 로또번호 당첨번호 검색)
```
    1197회(2025년 11월 08일 추첨)
    당첨번호 [1, 5, 7, 26, 28, 43]
    보너스 30
```

In [27]:
# 방법1
import requests
from bs4 import BeautifulSoup
url = 'https://dhlottery.co.kr/gameResult.do?method=byWin'
response = requests.get(url)
soup = BeautifulSoup(response.text, "html.parser")
#soup

In [28]:
# 방법2
from urllib.request import urlopen
url = 'https://dhlottery.co.kr/gameResult.do?method=byWin'
response = urlopen(url)
response.status
soup = BeautifulSoup(response, "html.parser")
# soup

In [29]:
times = soup.select_one('div.win_result strong').text # 1197회
date  = soup.select_one('div.win_result > p.desc').text
# date = soup.find('p', class_='desc').text
lotto_number_el = soup.select('div.num.win span')
lotto_number = [int(el.text) for el in lotto_number_el]
bonus_number = int(soup.select_one('div.num.bonus > p > span').text)

In [30]:
print(times, date)
print('당첨번호 ',lotto_number)
print('보 너 스 ', bonus_number)

1197회 (2025년 11월 08일 추첨)
당첨번호  [1, 5, 7, 26, 28, 43]
보 너 스  30


### 다음 검색 리스트
```
  no            title               href
  
  0     "한풀 꺾인 비트코인   https://v.daum.net/v/20251110094711892
```

In [31]:
# 방법1
import requests
from bs4 import BeautifulSoup
word = '비트코인'
url = 'https://search.daum.net/search?w=news&nil_search=btn&DA=PGD&enc=utf8&cluster=y&cluster_page=1&q='+word
print(url)
response = requests.get(url)
soup = BeautifulSoup(response.text, "html.parser")

items_find_list = [] # 검색한 결과를 담을 리스트(딕셔너리 list)
items_el = soup.select('div.item-title > strong.tit-g > a')
for idx, item in enumerate(items_el):
    # print(idx, item.text, item.attrs.get('href'))
    items_find_list.append({'no':idx,
                          'title':item.text,
                          'link':item.attrs.get('href')})
import pandas as pd
pd.DataFrame(items_find_list)

https://search.daum.net/search?w=news&nil_search=btn&DA=PGD&enc=utf8&cluster=y&cluster_page=1&q=비트코인


,no,title,link
0,0,"""한풀 꺾인 비트코인, 상승세 곧 회복""…근거는?",http://v.daum.net/v/20251110094711892
1,1,"""비트코인 21억 간다더니""...갑자기 말 바꿨다",http://v.daum.net/v/20251109105545139
2,2,비트코인 1억5500만원대 소폭 상승…트럼프 발언탓?,http://v.daum.net/v/20251110092814031
3,3,비트코인 '와르르' 내려오는데 '의외'의 전망,http://v.daum.net/v/20251107210115339
4,4,"“비트코인 약세, 닷컴버블과 판박이”…큰손·장투족 매일 던진다",http://v.daum.net/v/20251110094507791
5,5,"[가상자산 나침반] ‘겹악재’ 비트코인, 반전 포인트 나올까",http://v.daum.net/v/20251109112746909
6,6,"美 셧다운 종료 기대…비트코인, 10만 5000달러 회복[코인브리핑]",http://v.daum.net/v/20251110111435519
7,7,"비트플래닛, 비트코인 200개 매입 완료…자금 조달부터 커스터디까지 끝내",http://v.daum.net/v/20251110081500340
8,8,"""2000달러 관세 배당"" 비트코인에 불 지핀 트럼프…산타랠리 시작?",http://v.daum.net/v/20251110160448927
9,9,[비트코인 지금]가상화폐 흔들리자 급락한 코인 비축株,http://v.daum.net/v/20251110141113732


In [32]:
items_find_list = [] # 검색한 결과를 담을 리스트(딕셔너리 list)
items_el = soup.select('div.item-title > strong.tit-g > a')
for idx, item in enumerate(items_el):
    # print(idx, item.text, item.attrs.get('href'))
    items_find_list.append([idx, item.text, item.attrs['href']])
pd.DataFrame(items_find_list, columns=['번호', '제목','링크'])



,번호,제목,링크
0,0,"""한풀 꺾인 비트코인, 상승세 곧 회복""…근거는?",http://v.daum.net/v/20251110094711892
1,1,"""비트코인 21억 간다더니""...갑자기 말 바꿨다",http://v.daum.net/v/20251109105545139
2,2,비트코인 1억5500만원대 소폭 상승…트럼프 발언탓?,http://v.daum.net/v/20251110092814031
3,3,비트코인 '와르르' 내려오는데 '의외'의 전망,http://v.daum.net/v/20251107210115339
4,4,"“비트코인 약세, 닷컴버블과 판박이”…큰손·장투족 매일 던진다",http://v.daum.net/v/20251110094507791
5,5,"[가상자산 나침반] ‘겹악재’ 비트코인, 반전 포인트 나올까",http://v.daum.net/v/20251109112746909
6,6,"美 셧다운 종료 기대…비트코인, 10만 5000달러 회복[코인브리핑]",http://v.daum.net/v/20251110111435519
7,7,"비트플래닛, 비트코인 200개 매입 완료…자금 조달부터 커스터디까지 끝내",http://v.daum.net/v/20251110081500340
8,8,"""2000달러 관세 배당"" 비트코인에 불 지핀 트럼프…산타랠리 시작?",http://v.daum.net/v/20251110160448927
9,9,[비트코인 지금]가상화폐 흔들리자 급락한 코인 비축株,http://v.daum.net/v/20251110141113732


In [33]:
items_find_list = [] # 검색한 결과를 담을 리스트
# div.item-title > strong.tit-g > a
item_titles = soup.find_all('div', class_='item-title')
for idx, item in enumerate(item_titles):
    a = item.find('a')
    #print(idx, a.text, a.attrs['href'])
    #items_find_list.append([idx, a.text, a.attrs['href']])
    items_find_list.append({
        'no':idx,
        'title':a.text,
        'link':a.attrs['href'],
    })
pd.DataFrame(items_find_list)

,no,title,link
0,0,"""한풀 꺾인 비트코인, 상승세 곧 회복""…근거는?",http://v.daum.net/v/20251110094711892
1,1,"""비트코인 21억 간다더니""...갑자기 말 바꿨다",http://v.daum.net/v/20251109105545139
2,2,비트코인 1억5500만원대 소폭 상승…트럼프 발언탓?,http://v.daum.net/v/20251110092814031
3,3,비트코인 '와르르' 내려오는데 '의외'의 전망,http://v.daum.net/v/20251107210115339
4,4,"“비트코인 약세, 닷컴버블과 판박이”…큰손·장투족 매일 던진다",http://v.daum.net/v/20251110094507791
5,5,"[가상자산 나침반] ‘겹악재’ 비트코인, 반전 포인트 나올까",http://v.daum.net/v/20251109112746909
6,6,"美 셧다운 종료 기대…비트코인, 10만 5000달러 회복[코인브리핑]",http://v.daum.net/v/20251110111435519
7,7,"비트플래닛, 비트코인 200개 매입 완료…자금 조달부터 커스터디까지 끝내",http://v.daum.net/v/20251110081500340
8,8,"""2000달러 관세 배당"" 비트코인에 불 지핀 트럼프…산타랠리 시작?",http://v.daum.net/v/20251110160448927
9,9,[비트코인 지금]가상화폐 흔들리자 급락한 코인 비축株,http://v.daum.net/v/20251110141113732


In [34]:
# 다음 뉴스 검색(원하는 키워드를 원하는 페이지를 가져오기)
import requests
from bs4 import BeautifulSoup
import pandas as pd
def collect_news_list(keyword, page):
    'keyword로 page에 다음 뉴스검색한 결과를 출력->list를 return'
    #url = f'https://search.daum.net/search?w=news&nil_search=btn&DA=PGD&enc=utf8&cluster=y&cluster_page=1&q={keyword}&p={page}'
    # response = requests.get(url)
    url = f'https://search.daum.net/search?w=news&nil_search=btn&DA=PGD&enc=utf8'
    params = {'q':keyword, 'p':page}
    response = requests.get(url, params=params)
    soup = BeautifulSoup(response.text, 'html.parser')
    items_find_list = [] # 검색한 결과를 담는 리스트
    item_titles = soup.select('div.item-title > strong.tit-g > a')
    print(keyword)
    for idx, item in enumerate(item_titles):
           items_find_list.append([(page-1)*10+idx, item.text, item.attrs['href']])
    return items_find_list

In [35]:
collect_news_list('AI', 1)

AI


[[0,
  " 초중고 정보 교과 내 AI 교육 확대…모든 교육청 'AI 지원센터' 설립 ",
  'http://v.daum.net/v/20251110140139274'],
 [1,
  ' 초중고 AI교육 늘리고 ‘학·석·박사 5.5년’ 도입 ',
  'http://v.daum.net/v/20251110140447402'],
 [2,
  " 초중고부터 AI 기본교육 집중…'AI 박사' 5.5년 만에 키운다 ",
  'http://v.daum.net/v/20251110140128262'],
 [3,
  " PFCT, 'AI 신용평가 아카데미' 개최…AI 기술금융 인재 양성 ",
  'http://v.daum.net/v/20251110160614996'],
 [4,
  " [뉴스1 PICK]국내 최대 AI 컨펙스 'AI 서밋 서울 & 엑스포' 개막 ",
  'http://v.daum.net/v/20251110145713498'],
 [5,
  ' 5.5년만에 AI 박사학위 취득...내년 AI인재양성에 1.4조 예산 투입 ',
  'http://v.daum.net/v/20251110142138252'],
 [6,
  ' "도메인 지식 없는 AI는 공허"···최태원, 본업 기반 AI 경쟁력 강조 ',
  'http://v.daum.net/v/20251110144852927'],
 [7,
  ' 야놀자, 첫 AI 해커톤 개최…"AI 혁신 전사적 전파할 것" ',
  'http://v.daum.net/v/20251110160146766'],
 [8,
  " 중견회계법인협의회, AI 감사 솔루션 '오딘 AI' 도입 ",
  'http://v.daum.net/v/20251110150515967'],
 [9,
  ' [뉴욕증시] AI 거품론 속 급락…AI 선도주 엔비디아·팔란티어 폭락 ',
  'http://v.daum.net/v/20251105064248616']]

In [36]:
import time
keywords = ['청바지', '동대문']
result0 = [] # 청바지 1~3페이지까지 검색한 결과를 담을 list
result1 = [] # 동대문 1~3페이지까지 검색한 결과를 담을 list
pages = 3
for i, keyword in enumerate(keywords):
    print(f'==={i+1}번째 검색어 {keyword} 검색 결과 수집 중입니다===')
    for page in range(1, pages+1):
        if i==0:
            result0.extend(collect_news_list(keyword, page))
        elif i==1:
            result1.extend(collect_news_list(keyword, page))
        time.sleep(0.5)

===1번째 검색어 청바지 검색 결과 수집 중입니다===
청바지
청바지
청바지
===2번째 검색어 동대문 검색 결과 수집 중입니다===
동대문
동대문
동대문


In [37]:
result0_df = pd.DataFrame(result0, columns=['no','title','link'])
result1_df = pd.DataFrame(result1, columns=['no','title','link'])
result1_df.head()

,no,title,link
0,0,"“돈 벌어가세요” 동대문 일요시장, 겨울옷 핫플된 사연? [르포]",http://v.daum.net/v/20251110100357735
1,1,"동대문구, ‘11월 자전거교실’ 개강… 올해 마지막 초보자 교육 진행",http://v.daum.net/v/20251110145114101
2,2,"음주운전 차, 동대문역서 인도 돌진...日 관광객 '사망'",http://v.daum.net/v/20251103073316480
3,3,"동대문구, 25일까지 '주민 건강도시학교'···""건강하게 살자""",http://v.daum.net/v/20251109223318356
4,4,"서울 동대문구, 15일 배봉산정원 가족축제 개최",http://v.daum.net/v/20251104104517122


In [38]:
result0_df.to_csv(f'data/ch14_{keywords[0]}.csv', index=False)
result1_df.to_csv(f'data/ch14_{keywords[1]}.csv', index=False)

### 4) User-Agent를 추가하여 크롤링
- 자동으로 브라우저를 통해 요청하는 것 처럼 보이기

In [50]:
# 방법2 : 
from urllib.request import urlopen, Request
from urllib.parse import quote
word = quote('비트코인') # 한글을 encoding 처리
url = 'https://search.daum.net/search?w=news&nil_search=btn&DA=PGD&enc=utf8&cluster=y&cluster_page=1&q='+word
print(url)
headers = {'User-Agent':
          'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/142.0.0.0 Safari/537.36'}
# request = Request(url, headers=headers)
request = Request(url)
request.add_header('User-Agent',
                  'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/142.0.0.0 Safari/537.36')

response = urlopen(request)
soup = BeautifulSoup(response, 'html.parser')
# soup

https://search.daum.net/search?w=news&nil_search=btn&DA=PGD&enc=utf8&cluster=y&cluster_page=1&q=%EB%B9%84%ED%8A%B8%EC%BD%94%EC%9D%B8


- https://www.melon.com/chart/index.htm
    * https://www.melon.com/robots.txt 에서 Uesr-Agent에 봇이 지정

In [52]:
# 방법1
import requests
url = 'https://www.melon.com/chart/index.htm'
melonpage = requests.get(url)
melonpage
# soup = BeautifulSoup(melonpage.text, 'html.parser')
# soup 

In [58]:
# 방법2 : User-Agent 추가
from urllib.request import urlopen, Request
url = 'https://www.melon.com/chart/index.htm'
# melonpage = urlopen(url) # 에러남
headers = {'User-Agent':
          'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/142.0.0.0 Safari/537.36'}
request = Request(url, headers=headers)
melonpage = urlopen(request)
soup = BeautifulSoup(melonpage, 'html.parser')
# soup

In [70]:
 # 방법1 : User-Agent 추가
url = 'https://www.melon.com/chart/index.htm'
headers = {'user-agent':
          'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/142.0.0.0 Safari/537.36'}
melonpage = requests.get(url, headers=headers)
soup = BeautifulSoup(melonpage.text, # melonpage.content
                    'html.parser')

ranks = soup.select('td div.wrap.t_center > span.rank')
titles = soup.select('div.ellipsis.rank01 > span') # title.text.strip()
singers = soup.select('div.ellipsis.rank02') # singer.text.strip()[:20]
# 1위 | 노래제목 - 가수명
for rank, title, singer in zip(ranks, titles, singers):
    print(f'{rank.text}위 | {title.text.strip()} - {singer.text.strip()[:20]}')

1위 | Blue Valentine - NMIXXNMIXX
2위 | 타임캡슐 - 다비치다비치
3위 | Golden - HUNTR/X, EJAE, AUDRE
4위 | Drowning - WOODZWOODZ
5위 | Good Goodbye - 화사 (HWASA)화사 (HWASA)
6위 | SPAGHETTI (feat. j-hope of BTS) - LE SSERAFIM (르세라핌), 
7위 | 달리 표현할 수 없어요 - 로이킴로이킴
8위 | 뛰어(JUMP) - BLACKPINKBLACKPINK
9위 | Soda Pop - KPop Demon Hunters C
10위 | 어제보다 슬픈 오늘 - 우디 (Woody)우디 (Woody)
11위 | FAMOUS - ALLDAY PROJECTALLDAY
12위 | 시작의 아이 ❍ - 박다혜, 마크툽 (MAKTUB)박다혜
13위 | IRIS OUT - Kenshi YonezuKenshi 
14위 | XOXZ - IVE (아이브)IVE (아이브)
15위 | 모르시나요(PROD.로코베리) - 조째즈조째즈
16위 | Rich Man - aespaaespa
17위 | Hollywood Action - BOYNEXTDOORBOYNEXTDO
18위 | 한번 더 이별 - 이창섭이창섭
19위 | 내게 사랑이 뭐냐고 물어본다면 - 로이킴로이킴
20위 | 너에게 닿기를 - 10CM10CM
21위 | HOME SWEET HOME (feat. 태양, 대성) - G-DRAGONG-DRAGON
22위 | 운명 (2025) - 먼데이 키즈, 이이경먼데이 키즈, 이
23위 | body - 다영 (DAYOUNG)다영 (DAYO
24위 | toxic till the end - 로제 (ROSÉ)로제 (ROSÉ)
25위 | Whiplash - aespaaespa
26위 | 시작의 아이 - 마크툽 (MAKTUB)마크툽 (MAK
27위 | 천상연 - 이창섭이창섭
28위 | 나는 반딧불 - 황가람황가람
29위 | 어떻게 이별까지 사랑하겠어, 널 사랑하는 거지 - AK